---
cdt: 2024-10-30T20:29:58
title: Optimizing Parafac2 Parameters
description: now that parafac2 is prototyped, need to find optimal parameters
---


So far we've consistently had negative components and multipeak components. The expectation is that no components will be negative, and there will be one peak per component. Furthermore all components will approximate a gaussian peak.


In [ ]:
%reload_ext autoreload
%autoreload 2

# get the test data as two tables: metadata and a samplewise stacked img table
import logging

import duckdb as db

import polars as pl

from pca_analysis.definitions import DB_PATH_UV
from pca_analysis.notebooks.experiments.parafac2_pipeline.parafac2_decomposition import (
    get_input_data,
)

from pca_analysis.code.get_sample_data import get_ids_by_varietal
import plotly.express as px

logger = logging.getLogger(__name__)

logger.setLevel(logging.DEBUG)


with db.connect(DB_PATH_UV) as conn:
    ids = get_ids_by_varietal("shiraz", conn)

ids


['82', '84', '68', '0151', '115', '0101', '39', '111', '61', '62']

In [ ]:
input_data = get_input_data(DB_PATH_UV, ids)


In [ ]:
input_data.filter_nm_tbl(pl.col("mins").le(4.55)).plot_2d_facet(
    256, "shiraz dataset @ 256", 4
)


As we can see all signals bar 0101 are on a similar scale. I will elect to exclude it from the test set.


In [ ]:
input_data = input_data.filter_nm_tbl(
    (~pl.col("runid").eq("0101")).and_(pl.col("mins").le(4.55))
)


In [ ]:
X = input_data.to_X()


## CORCONDIA

We will attempt to estimate the rank via CORCONDIA.
